In [1]:
import requests
import csv
import pandas as pd

In [2]:
url='http://compras.dados.gov.br/compraSemLicitacao/v1/compras_slicitacao.csv'
ano = '2018'
resultado = pd.DataFrame()

In [3]:
def convertRequestToCsv(response):
    decoded_content = response.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    df = pd.DataFrame(my_list)
    ##Sei que não é o certo, mas é o que pude resolver no momento
    colunas= df.iloc[0]
    novo  = pd.DataFrame(df.values[1:], columns=colunas)
    return novo

In [4]:
def listarLicitacoes(valor, url):
    frames = []
    for x in range(0, valor, 500):
        payload = {'dt_ano_aviso_licitacao': ano, 'offset': x}
        response = requests.get(url, params=payload)
        data = convertRequestToCsv(response)
        frames.append(data)
    result = pd.concat(frames)
    result = result[['Orgão','UASG', 'Código da modalidade da compra', 'Número processo', 
                     'Quantidade item','Descrição objeto licitação','Itens > uri'] ]
    
    for index, row in result.iterrows():
            licitacao = str(row['Itens > uri'])
            descartar, licitacao = licitacao.split('_slicitacao/')
            licitacao, descartar = licitacao.split('/')
            result.loc[ index, 'Número da Licitação' ] = str( licitacao)
            
    return result

In [5]:
def listaItensLicitacao(licitacao, numeroLicitacao):
    link = 'http://compras.dados.gov.br'+licitacao+'.csv'
    lista = pd.read_csv(link)
    lista['Número da Licitação'] = numeroLicitacao
    lista = lista[['Número da Licitação', 'Descrição do item - Serviço', 'Quantidade do item',
                   'Tipo do fornecedor', 'Pessoa Jurídica Contratada']]
    return lista

In [6]:
def concatenacaoListasItensLicitacao(data):
    listaItens = []
    for index, row in data.iterrows():
        try:
            lista = listaItensLicitacao(str(row['Itens > uri']), str(row['Número da Licitação']))
            listaItens.append(lista)
        except ValueError:
            # ainda para ajeitar, questao de tempo
            lista2 = pd.DataFrame()
            lista2.loc[ index, 'Número da Licitação' ] = str( row['Número da Licitação'])
            listaItens.append(lista2)
            print('error')
        print(index)
    result = pd.concat(listaItens)
    return result

In [7]:
def listaCompleta(data,resultado):
    juncao = pd.merge(data,resultado,on='Número da Licitação')
    return juncao

In [8]:
data = listarLicitacoes(valor=500, url=url)

In [9]:
data = data[:110]
data.head()

,Orgão,UASG,Código da modalidade da compra,Número processo,Quantidade item,Descrição objeto licitação,Itens > uri,Número da Licitação
0,"26427: INST.FED.DE EDUC.,CIENC.E TECNOLOGIA DA...",158403: INST.FED.DA BAHIA/CAMPUS CAMAÇARI,7: Inexigibilidade,23285000067201838,,Objeto: Fornecimento de Água e Coleta de Esgot...,/compraSemLicitacao/id/item_slicitacao/1584030...,15840307000012018
1,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018
2,"26406: INST.FED.DE EDUC.,CIENC.E TEC. DO ESP.S...",158428: IFES - CAMPUS IBATIBA,7: Inexigibilidade,23184000001320184,,Objeto: Contratação de serviços do Departament...,/compraSemLicitacao/id/item_slicitacao/1584280...,15842807000012018
3,20701: INST.BRAS.DO MEIO AMB.E DOS REC.NAT.REN...,193120: IBAMA - SUPERINTENDENCIA ESTADUAL/RN,7: Inexigibilidade,02021.100979/2017,,Objeto: Primeiro Termo Aditivo para Prorrogaçã...,/compraSemLicitacao/id/item_slicitacao/1931200...,19312007000012018
4,"26417: INST.FED.DE EDUC.,CIENC.E TEC. DA PARAIBA",155890: IFECT DA PARAÍBA - CAMPUS ITAPORANGA,6: Dispensa,23796000066201823,,Objeto: Dispensa de licitação baseada no incis...,/compraSemLicitacao/id/item_slicitacao/1558900...,15589006000012018


In [10]:
resultado = concatenacaoListasItensLicitacao(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
error
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


/home/eduardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [11]:
juncao = listaCompleta(data,resultado)
juncao.head(10)

,Orgão,UASG,Código da modalidade da compra,Número processo,Quantidade item,Descrição objeto licitação,Itens > uri,Número da Licitação,Descrição do item - Serviço,Pessoa Jurídica Contratada,Quantidade do item,Tipo do fornecedor
0,"26427: INST.FED.DE EDUC.,CIENC.E TECNOLOGIA DA...",158403: INST.FED.DA BAHIA/CAMPUS CAMAÇARI,7: Inexigibilidade,23285000067201838,,Objeto: Fornecimento de Água e Coleta de Esgot...,/compraSemLicitacao/id/item_slicitacao/1584030...,15840307000012018,FORNECIMENTO DE AGUA E COLETA DE ESGOTO SANITARIO,Fornecedor 13.504.675/0001-10: 13504675000110,1.0,Pessoa Jurídica
1,"26427: INST.FED.DE EDUC.,CIENC.E TECNOLOGIA DA...",158403: INST.FED.DA BAHIA/CAMPUS CAMAÇARI,7: Inexigibilidade,23285000067201838,,Objeto: Fornecimento de Água e Coleta de Esgot...,/compraSemLicitacao/id/item_slicitacao/1584030...,15840307000012018,FORNECIMENTO DE AGUA E COLETA DE ESGOTO SANITARIO,Fornecedor 13.504.675/0001-10: 13504675000110,1.0,Pessoa Jurídica
2,"26427: INST.FED.DE EDUC.,CIENC.E TECNOLOGIA DA...",158403: INST.FED.DA BAHIA/CAMPUS CAMAÇARI,7: Inexigibilidade,23285000067201838,,Objeto: Fornecimento de Água e Coleta de Esgot...,/compraSemLicitacao/id/item_slicitacao/1584030...,15840307000012018,FORNECIMENTO DE AGUA E COLETA DE ESGOTO SANITARIO,Fornecedor 13.504.675/0001-10: 13504675000110,1.0,Pessoa Jurídica
3,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,10.0,Pessoa Jurídica
4,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,10.0,Pessoa Jurídica
5,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,10.0,Pessoa Jurídica
6,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,10.0,Pessoa Jurídica
7,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,4.0,Pessoa Jurídica
8,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,4.0,Pessoa Jurídica
9,52121: COMANDO DO EXERCITO,160514: COUDELARIA DE RINCAO/RS,7: Inexigibilidade,00401000270201835,,Objeto: Atender Despesas com Aquisição de Mate...,/compraSemLicitacao/id/item_slicitacao/1605140...,16051407000012018,NaN,Fornecedor 07.944.389/0006-87: 07944389000687,4.0,Pessoa Jurídica
